In [146]:
import tensorflow as tf 
import tensorflow_probability as tfp
from tensorflow.keras.datasets import mnist

In [147]:
import numpy as np

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns

import argparse
import random
import os
import math


tfd = tfp.distributions

In [148]:
import importlib
import new_custom_layers
import utilities

In [149]:
# Hyperparameter setup
learning_rate = 0.001
epochs = 100
batch_size = 128
num_classes = 10
num_samples = 1

### MNIST data (5000 train, 5000 val and 10000 test) with 50% noise in training data

In [150]:
# # Load and preprocess MNIST data
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# x_train = x_train.reshape([-1, 28*28])
# x_test = x_test.reshape([-1, 28*28])
# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255

# y_train = y_train.astype('int32')
# y_test = y_test.astype('int32')

# train_samples = x_train.shape[0]
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')

# x_val, y_val = x_test, y_test
n_samples = 5000
I = np.eye(num_classes)
x_train_S, y_train_S, x_test, y_test = utilities.load_mnist(n_samples, square=False, conv=False)
y_train_S = y_train_S.astype('int32')
y_test = y_test.astype('int32')

N = int(n_samples/2)

x_val = x_train_S[N:]
y_val = y_train_S[N:]

x_train = x_train_S[:N]
y_train = y_train_S[:N]

# Add output noise to important class
R = int(.5*N)
r_ind = random.sample(range(0,N),R)
r_class = np.random.randint(0,num_classes,R)
y_train[r_ind] = r_class

train_samples = N

x_train shape: (5000, 784)
5000 train samples
10000 test samples


### Loss matrix 

In [31]:
digits = [3,8]
np.set_printoptions(precision=3)
L = np.copy(I)
L[np.where(I==0)] = 1
# L[[3,8]] = 0.7 # Select more important rows with lower loss in prediction
L[np.where(I==1)] = 0

loss_mat = L
M = 1.0001

print("         TRUTH\n" )
string = 'PRED'
for i in range(num_classes):
    if i < 4:
        print(string[i],'  ',str(i),': ',M - loss_mat[i])
    else:
        print('    ',str(i),': ',M - loss_mat[i])
        
# Set up weight for weighted cross entropy

class_weight = np.ones((num_classes))
class_weight[digits] = 1.4

print(loss_mat)

         TRUTH

P    0 :  [1.e+00 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04]
R    1 :  [1.e-04 1.e+00 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04]
E    2 :  [1.e-04 1.e-04 1.e+00 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04]
D    3 :  [1.e-04 1.e-04 1.e-04 1.e+00 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04]
     4 :  [1.e-04 1.e-04 1.e-04 1.e-04 1.e+00 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04]
     5 :  [1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e+00 1.e-04 1.e-04 1.e-04 1.e-04]
     6 :  [1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e+00 1.e-04 1.e-04 1.e-04]
     7 :  [1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e+00 1.e-04 1.e-04]
     8 :  [1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e+00 1.e-04]
     9 :  [1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e-04 1.e+00]
[[0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 0. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 0. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 0. 1. 1. 1. 1. 1.]

In [33]:
import custom_layers

### Softmax loss

In [34]:
# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.
scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)

x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')

logits = custom_layers.Softmax( 
    10, 
    kernel_divergence_fn=scaled_kl)([x_input, y_input])

model_sm = tf.keras.Model([x_input, y_input], logits)
model_sm.compile(loss=None, optimizer='adam')

In [35]:

history_model_sm = []
for epoch in range(epochs):
    history_epoch = model_sm.fit([x_train, y_train],
                 batch_size=batch_size,
                 epochs=1,
                 verbose=1,
                 validation_data=([x_val, y_val], None))
#                  callbacks=[model_checkpoint])
#     T = 10
#     yt_hat = np.array([lc_model.predict([x_train,H_x]) for _ in range(T)])
#     H_x = lcbnn.optimal_h(yt_hat,loss_mat)
#     h_train = np.argmax(model.predict([x_train, y_train, h_train]), -1)
    history_model_sm.append(history_epoch)
       

Train on 2500 samples, validate on 2500 samples
2500/2500 [==============================] - 1s 276us/sample - loss: 10.5079 - val_loss: 10.2027
Train on 2500 samples, validate on 2500 samples
2500/2500 [==============================] - 0s 32us/sample - loss: 10.1920 - val_loss: 9.8040
Train on 2500 samples, validate on 2500 samples
2500/2500 [==============================] - 0s 36us/sample - loss: 10.0016 - val_loss: 9.5019
Train on 2500 samples, validate on 2500 samples
2500/2500 [==============================] - 0s 48us/sample - loss: 9.8761 - val_loss: 9.3110
Train on 2500 samples, validate on 2500 samples
2500/2500 [==============================] - 0s 31us/sample - loss: 9.7879 - val_loss: 9.1557
Train on 2500 samples, validate on 2500 samples
2500/2500 [==============================] - 0s 29us/sample - loss: 9.7019 - val_loss: 9.0630
Train on 2500 samples, validate on 2500 samples
2500/2500 [==============================] - 0s 47us/sample - loss: 9.6267 - val_loss: 8.9494
T

2500/2500 [==============================] - 0s 51us/sample - loss: 7.2715 - val_loss: 6.7213
Train on 2500 samples, validate on 2500 samples
2500/2500 [==============================] - 0s 61us/sample - loss: 7.2220 - val_loss: 6.6944
Train on 2500 samples, validate on 2500 samples
2500/2500 [==============================] - 0s 53us/sample - loss: 7.1878 - val_loss: 6.6457
Train on 2500 samples, validate on 2500 samples
2500/2500 [==============================] - 0s 143us/sample - loss: 7.1463 - val_loss: 6.5962
Train on 2500 samples, validate on 2500 samples
2500/2500 [==============================] - 0s 84us/sample - loss: 7.1007 - val_loss: 6.5741
Train on 2500 samples, validate on 2500 samples
2500/2500 [==============================] - 0s 39us/sample - loss: 7.0877 - val_loss: 6.5408
Train on 2500 samples, validate on 2500 samples
2500/2500 [==============================] - 0s 29us/sample - loss: 7.0350 - val_loss: 6.4941
Train on 2500 samples, validate on 2500 samples
2500/

In [ ]:
y_pred = np.argmax(model_sm.predict([x_test,y_test]), 1)
print("Test accuracy: ", len(np.where(y_pred - y_test == 0)[0])/len(y_test))

y_pred = np.argmax(model_sm.predict([x_train, y_train]), 1)
print("Train accuracy: ", len(np.where(y_pred - y_train == 0)[0])/len(y_train))

### Loss calibrated softmax loss

In [ ]:
# For reproduceable results
importlib.reload(custom_layers)
importlib.reload(utilities)

# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.

scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)

x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')
h_input =  tf.keras.Input((1,), name = 'h_input')

logits = custom_layers.LCSoftmax( 
    10, 
    M,
    loss_mat,
    kernel_divergence_fn=scaled_kl)([x_input, y_input, h_input])

model_sm_lc = tf.keras.Model([x_input, y_input, h_input], logits)
model_sm_lc.compile(loss=None, optimizer='adam')

In [ ]:
h_train = y_train
h_test = y_test
h_val = y_val

history_model_sm_lc = []
for epoch in range(epochs):
    history_epoch = model_sm_lc.fit([x_train, y_train, h_train],
                 batch_size=batch_size,
                 epochs=1,
                 verbose=1,
                 validation_data=([x_val, y_val, h_val], None))
#                  callbacks=[model_checkpoint])
    T = 10
    yt_hat = np.array([model_sm_lc.predict([x_train, y_train, h_train]) for _ in range(T)])
    h_train = utilities.optimal_h(yt_hat, loss_mat)
    history_model_sm_lc.append(history_epoch)
       

### One-vs-each loss

In [ ]:
# For reproduceable results
importlib.reload(custom_layers)
importlib.reload(utilities)
# np.random.seed(1337)

# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.

scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)


x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')

logits = custom_layers.OVELayer(
    5,
    10, 
    kernel_divergence_fn=scaled_kl)([x_input, y_input])

model_ove = tf.keras.Model([x_input, y_input], logits)
model_ove.compile(loss=None, optimizer='adam')

In [ ]:

history_model_ove = []
for epoch in range(epochs):
    history_epoch = model_ove.fit([x_train, y_train],
                 batch_size=batch_size,
                 epochs=1,
                 verbose=1,
                 validation_data=([x_val, y_val], None))
#                  callbacks=[model_checkpoint])

    history_model_ove.append(history_epoch)
       

In [ ]:
# For reproduceable results
importlib.reload(custom_layers)
importlib.reload(utilities)
# np.random.seed(1337)

# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.

scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)


x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')

logits = custom_layers.NCE(
    5,
    10, 
    kernel_divergence_fn=scaled_kl)([x_input, y_input])

model_nce = tf.keras.Model([x_input, y_input], logits)
model_nce.compile(loss=None, optimizer='adam')

In [ ]:

history_model_nce = []
for epoch in range(200):
    history_epoch = model_nce.fit([x_train, y_train],
                 batch_size=batch_size,
                 epochs=1,
                 verbose=1,
                 validation_data=([x_val, y_val], None))
#                  callbacks=[model_checkpoint])

    history_model_nce.append(history_epoch)
       

## Risk evaluation

In [ ]:
#  Softmax model

T = 10
yt_hat_std = np.array([model_sm.predict([x_test, y_test]) for _ in range(T)])
H_x_test_std = utilities.optimal_h(yt_hat_std,loss_mat) 
acc_std = accuracy_score(y_test, H_x_test_std)
loss_std = np.mean(loss_mat[y_test, H_x_test_std])
print('Standard:\n')
print('Accuracy on optimal decision: ', acc_std)
print('Expected loss: ', loss_std)


# #  Softmax with loss calibration

# T = 10
# yt_hat_sm_lc = np.array([model_sm_lc.predict([x_test, y_test, h_test]) for _ in range(T)])
# H_x_test_sm_lc = utilities.optimal_h(yt_hat_sm_lc,loss_mat) 
# acc_sm_lc = accuracy_score(y_test, H_x_test_sm_lc)
# loss_sm_lc = np.mean(loss_mat[y_test, H_x_test_sm_lc])
# print('\nLoss calibrated softmax:\n')
# print('Accuracy on optimal decision: ', acc_sm_lc)
# print('Expected loss: ', loss_sm_lc)


# OVE model

T = 10 
yt_hat_ove = np.array([model_ove.predict([x_test, y_test]) for _ in range(T)])
H_x_test_ove = utilities.optimal_h(yt_hat_ove,loss_mat) 
acc_ove = accuracy_score(y_test, H_x_test_ove)
loss_ove = np.mean(loss_mat[y_test, H_x_test_ove])
print('\nOVE:\n')
print('Accuracy on optimal decision: ', acc_ove)
print('Expected loss: ', loss_ove)



# NCE model

T = 10 
yt_hat_nce = np.array([model_nce.predict([x_test, y_test]) for _ in range(T)])
H_x_test_nce = utilities.optimal_h(yt_hat_nce,loss_mat) 
acc_nce = accuracy_score(y_test, H_x_test_nce)
loss_nce = np.mean(loss_mat[y_test, H_x_test_nce])
print('\nNCE:\n')
print('Accuracy on optimal decision: ', acc_nce)
print('Expected loss: ', loss_nce)


In [ ]:
plt.rcParams['figure.figsize'] = (10, 10) # Width and height

classes = ['0','1','2','3','4','5','6','7','8','9']


cnf_matrix_std = confusion_matrix(y_test.astype('float'),np.round(H_x_test_std))
np.set_printoptions(precision=2)
cnf_matrix_sm_lc = confusion_matrix(y_test.astype('float'),np.round(H_x_test_sm_lc))
np.set_printoptions(precision=2)
cnf_matrix_ove = confusion_matrix(y_test.astype('float'),np.round(H_x_test_ove))
np.set_printoptions(precision=2)
cnf_matrix_nce = confusion_matrix(y_test.astype('float'),np.round(H_x_test_nce))
np.set_printoptions(precision=2)

plt.subplot(221)
utilities.plot_confusion_matrix(cnf_matrix_std, classes=classes, normalize=True,
                      title='Softmax')
plt.subplot(222)
utilities.plot_confusion_matrix(cnf_matrix_sm_lc, classes=classes, normalize=True,
                      title='LC Softmax')
plt.subplot(223)
utilities.plot_confusion_matrix(cnf_matrix_ove, classes=classes, normalize=True,
                      title='OVE')
plt.subplot(224)
utilities.plot_confusion_matrix(cnf_matrix_nce, classes=classes, normalize=True,
                      title='NCE')

In [8]:
digits = [3,8]
np.set_printoptions(precision=3)
L = np.copy(I)
L[np.where(I==0)] = 1
L[[3,8]] = 0.7 # Select more important rows with lower loss in prediction
L[np.where(I==1)] = 0

loss_mat = L
M = 1.000

print("         TRUTH\n" )
string = 'PRED'
util_mat = M - loss_mat
for i in range(num_classes):
    if i < 4:
        print(string[i],'  ',str(i),': ',M - loss_mat[i])
    else:
        print('    ',str(i),': ',M - loss_mat[i])
        
# Set up weight for weighted cross entropy

class_weight = np.ones((num_classes))
class_weight[digits] = 1.4

print(loss_mat)

         TRUTH

P    0 :  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
R    1 :  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
E    2 :  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
D    3 :  [0.3 0.3 0.3 1.  0.3 0.3 0.3 0.3 0.3 0.3]
     4 :  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
     5 :  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
     6 :  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
     7 :  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
     8 :  [0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 1.  0.3]
     9 :  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[[0.  1.  1.  1.  1.  1.  1.  1.  1.  1. ]
 [1.  0.  1.  1.  1.  1.  1.  1.  1.  1. ]
 [1.  1.  0.  1.  1.  1.  1.  1.  1.  1. ]
 [0.7 0.7 0.7 0.  0.7 0.7 0.7 0.7 0.7 0.7]
 [1.  1.  1.  1.  0.  1.  1.  1.  1.  1. ]
 [1.  1.  1.  1.  1.  0.  1.  1.  1.  1. ]
 [1.  1.  1.  1.  1.  1.  0.  1.  1.  1. ]
 [1.  1.  1.  1.  1.  1.  1.  0.  1.  1. ]
 [0.7 0.7 0.7 0.7 0.7 0.7 0.7 0.7 0.  0.7]
 [1.  1.  1.  1.  1.  1.  1.  1.  1.  0. ]]


In [200]:
import new_custom_layers
import custom_layers

### Stochastic loss calibration for softmax

In [719]:
# For reproduceable results
importlib.reload(new_custom_layers)
importlib.reload(utilities)

# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.

scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)

x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')

outputs = x_input
logits = new_custom_layers.DUCARLayer(
    5,
    num_classes, 
    util_mat,
    kernel_divergence_fn=scaled_kl)([outputs, y_input])

model_sm_slc = tf.keras.Model([x_input, y_input], logits)
model_sm_slc.compile(loss=None, optimizer='adam')

In [724]:
# history_model_sm_slc = []
# for epoch in range(10):
history_epoch = model_sm_slc.fit([x_train, y_train],
             batch_size=batch_size,
             epochs=10,
             verbose=1)
#                  validation_data=([x_val, y_val, h_val], None))
#                  callbacks=[model_checkpoint])
#     T = 1
#     yt_hat = np.array([model_sm_slc.predict([x_train, y_train, h_train]) for _ in range(T)])
#     h_train = utilities.optimal_h(yt_hat, loss_mat)
#     history_model_sm_slc.append(history_epoch)
       
#     h_train = model_sm_slc.predict([x_train, y_train, h_train])

Epoch 1/10
2500/2500 [==============================] - 0s 47us/step - loss: 9.7865
Epoch 2/10
2500/2500 [==============================] - 0s 59us/step - loss: 9.8160
Epoch 3/10
2500/2500 [==============================] - 0s 52us/step - loss: 9.7189
Epoch 4/10
2500/2500 [==============================] - 0s 49us/step - loss: 9.7049
Epoch 5/10
2500/2500 [==============================] - 0s 53us/step - loss: 9.6569
Epoch 6/10
2500/2500 [==============================] - 0s 46us/step - loss: 9.5859
Epoch 7/10
2500/2500 [==============================] - 0s 39us/step - loss: 9.5106
Epoch 8/10
2500/2500 [==============================] - 0s 49us/step - loss: 9.4889
Epoch 9/10
2500/2500 [==============================] - 0s 45us/step - loss: 9.5324
Epoch 10/10
2500/2500 [==============================] - 0s 57us/step - loss: 9.4509


In [725]:
#  Softmax with stochastic loss calibration
T = 10
yt_hat_sm_slc = np.array([np.exp(model_sm_slc.predict([x_test, y_test])) for _ in range(T)])
H_x_test_sm_slc = utilities.optimal_h(yt_hat_sm_slc,loss_mat) 
acc_sm_slc = accuracy_score(y_test, H_x_test_sm_slc)
loss_sm_slc = np.mean(loss_mat[y_test, H_x_test_sm_slc])
print('\nLoss calibrated softmax:\n')
print('Accuracy on optimal decision: ', acc_sm_slc)
print('Expected loss: ', loss_sm_slc)


Loss calibrated softmax:

Accuracy on optimal decision:  0.7352
Expected loss:  0.24700999999999998


In [394]:
importlib.reload(new_custom_layers)
importlib.reload(utilities)


# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.
scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)

x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')

outputs = x_input
# output = tfp.layers.DenseReparameterization(50,
#             activation = tf.nn.relu,
#             kernel_divergence_fn=scaled_kl)(x_input)
logits = custom_layers.DUCARLayer( 
    5,
    num_classes, 
    M,
    util_mat,
    kernel_divergence_fn=scaled_kl)([outputs, y_input])

model_sm = tf.keras.Model([x_input, y_input], logits)
model_sm.compile(loss=None, optimizer='adam')

ValueError: Layer ducar_layer_10 expects 3 inputs, but it received 2 input tensors. Inputs received: [<tf.Tensor 'x_input_52:0' shape=(?, 784) dtype=float32>, <tf.Tensor 'y_input_52:0' shape=(?, 1) dtype=float32>]

In [359]:

history_model_sm = []
for epoch in range(10):
    history_epoch = model_sm.fit([x_train, y_train],
                 batch_size=batch_size,
                 epochs=1,
                 verbose=1)
    history_model_sm.append(history_epoch)
       

Epoch 1/1
2500/2500 [==============================] - 0s 96us/step - loss: 7.2504
Epoch 1/1
2500/2500 [==============================] - 0s 142us/step - loss: 7.2307
Epoch 1/1
2500/2500 [==============================] - 0s 129us/step - loss: 7.1590
Epoch 1/1
2500/2500 [==============================] - 0s 161us/step - loss: 7.1574
Epoch 1/1
2500/2500 [==============================] - 1s 206us/step - loss: 7.0899 0s - loss: 7.
Epoch 1/1
2500/2500 [==============================] - 1s 242us/step - loss: 7.0519
Epoch 1/1
2500/2500 [==============================] - 0s 136us/step - loss: 7.0131
Epoch 1/1
2500/2500 [==============================] - 1s 222us/step - loss: 6.9944
Epoch 1/1
2500/2500 [==============================] - 1s 207us/step - loss: 6.9820
Epoch 1/1
2500/2500 [==============================] - 0s 124us/step - loss: 6.9140


In [370]:
T = 10
yt_hat_std = np.array([np.exp(model_sm.predict([x_test, y_test])) for _ in range(T)])
H_x_test_std = utilities.optimal_h(yt_hat_std,loss_mat) 
acc_std = accuracy_score(y_test, H_x_test_std)
loss_std = np.mean(loss_mat[y_test, H_x_test_std])
print('Standard:\n')
print('Accuracy on optimal decision: ', acc_std)
print('Expected loss: ', loss_std)

Standard:

Accuracy on optimal decision:  0.79
Expected loss:  0.21


In [361]:
loss_mat

array([[0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 0., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 0., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 0., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 0., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 0., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 0., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 0., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.]])

In [362]:
# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.
importlib.reload(custom_layers)
importlib.reload(utilities)

scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)

x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')

outputs = x_input
# output = tfp.layers.DenseReparameterization(50,
#             activation = tf.nn.relu,
#             kernel_divergence_fn=scaled_kl)(x_input)
logits = custom_layers.ARLayer(
    5,
    10,
    kernel_divergence_fn=scaled_kl)([outputs, y_input])

model_ar = tf.keras.Model([x_input, y_input], logits)
model_ar.compile(loss=None, optimizer='adam')

In [367]:

history_model_ar = []
for epoch in range(10):
    history_epoch = model_ar.fit([x_train, y_train],
                 batch_size=batch_size,
                 epochs=1,
                 verbose=1)
#                  validation_data=([x_val, y_val], None))
#                  callbacks=[model_checkpoint])
#     T = 10
#     yt_hat = np.array([lc_model.predict([x_train,H_x]) for _ in range(T)])
#     H_x = lcbnn.optimal_h(yt_hat,loss_mat)
#     h_train = np.argmax(model.predict([x_train, y_train, h_train]), -1)
    history_model_ar.append(history_epoch)
       

Epoch 1/1
2500/2500 [==============================] - 0s 91us/step - loss: 7.2256
Epoch 1/1
2500/2500 [==============================] - 0s 122us/step - loss: 7.2233
Epoch 1/1
2500/2500 [==============================] - 0s 94us/step - loss: 7.1472
Epoch 1/1
2500/2500 [==============================] - 0s 111us/step - loss: 7.0855
Epoch 1/1
2500/2500 [==============================] - 0s 193us/step - loss: 7.1392
Epoch 1/1
2500/2500 [==============================] - 0s 120us/step - loss: 7.0206
Epoch 1/1
2500/2500 [==============================] - 0s 117us/step - loss: 7.0012
Epoch 1/1
2500/2500 [==============================] - 0s 98us/step - loss: 6.9478
Epoch 1/1
2500/2500 [==============================] - 0s 113us/step - loss: 6.9387
Epoch 1/1
2500/2500 [==============================] - 0s 101us/step - loss: 6.9261


In [369]:
T = 10
yt_hat_ar = np.array([(model_ar.predict([x_test, y_test])) for _ in range(T)])
H_x_test_ar = utilities.optimal_h(yt_hat_ar,loss_mat) 
acc_ar = accuracy_score(y_test, H_x_test_ar)
loss_ar = np.mean(loss_mat[y_test, H_x_test_ar])
print('Standard:\n')
print('Accuracy on optimal decision: ', acc_std)
print('Expected loss: ', loss_ar)

Standard:

Accuracy on optimal decision:  0.7929
Expected loss:  0.2409


In [ ]:
tf.RaggedTensor

In [47]:
# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.
scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)

x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')

logits = new_custom_layers.ARLayer( 
    5,
    10, 
    kernel_divergence_fn=scaled_kl)([x_input, y_input])

model_sm = tf.keras.Model([x_input, y_input], logits)
model_sm.compile(loss=None, optimizer='adam')

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


In [56]:

history_model_sm = []
for epoch in range(10):
    history_epoch = model_sm.fit([x_train, y_train],
                 batch_size=batch_size,
                 epochs=1,
                 verbose=1)
#                  validation_data=([x_val, y_val], None))
#                  callbacks=[model_checkpoint])
#     T = 10
#     yt_hat = np.array([lc_model.predict([x_train,H_x]) for _ in range(T)])
#     H_x = lcbnn.optimal_h(yt_hat,loss_mat)
#     h_train = np.argmax(model.predict([x_train, y_train, h_train]), -1)
    history_model_sm.append(history_epoch)
       

Epoch 1/1
2500/2500 [==============================] - 0s 27us/step - loss: 7.9778
Epoch 1/1
2500/2500 [==============================] - 0s 40us/step - loss: 7.9859
Epoch 1/1
2500/2500 [==============================] - 0s 34us/step - loss: 7.8957
Epoch 1/1
2500/2500 [==============================] - 0s 40us/step - loss: 7.8621
Epoch 1/1
2500/2500 [==============================] - 0s 37us/step - loss: 7.8085
Epoch 1/1
2500/2500 [==============================] - 0s 41us/step - loss: 7.7841
Epoch 1/1
2500/2500 [==============================] - 0s 34us/step - loss: 7.7173
Epoch 1/1
2500/2500 [==============================] - 0s 39us/step - loss: 7.6742
Epoch 1/1
2500/2500 [==============================] - 0s 25us/step - loss: 7.6717
Epoch 1/1
2500/2500 [==============================] - 0s 43us/step - loss: 7.6741


In [57]:
T = 10
yt_hat_std = np.array([np.exp(model_sm.predict([x_test, y_test])) for _ in range(T)])
H_x_test_std = utilities.optimal_h(yt_hat_std,loss_mat) 
acc_std = accuracy_score(y_test, H_x_test_std)
loss_std = np.mean(loss_mat[y_test, H_x_test_std])
print('Standard:\n')
print('Accuracy on optimal decision: ', acc_std)
print('Expected loss: ', loss_std)

Standard:

Accuracy on optimal decision:  0.6128
Expected loss:  0.38081000000000004


In [372]:
# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.
scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)

x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')

logits = new_custom_layers.Softmax( 
#     5,
    10, 
    kernel_divergence_fn=scaled_kl)([x_input, y_input])

model_sm = tf.keras.Model([x_input, y_input], logits)
model_sm.compile(loss=None, optimizer='adam')

In [381]:

history_model_sm = []
for epoch in range(10):
    history_epoch = model_sm.fit([x_train, y_train],
                 batch_size=batch_size,
                 epochs=1,
                 verbose=1)
#                  validation_data=([x_val, y_val], None))
#                  callbacks=[model_checkpoint])
#     T = 10
#     yt_hat = np.array([lc_model.predict([x_train,H_x]) for _ in range(T)])
#     H_x = lcbnn.optimal_h(yt_hat,loss_mat)
#     h_train = np.argmax(model.predict([x_train, y_train, h_train]), -1)
    history_model_sm.append(history_epoch)
       

Epoch 1/1
2500/2500 [==============================] - 0s 88us/step - loss: 8.3663
Epoch 1/1
2500/2500 [==============================] - 0s 77us/step - loss: 8.3486
Epoch 1/1
2500/2500 [==============================] - 0s 114us/step - loss: 8.2715
Epoch 1/1
2500/2500 [==============================] - 0s 182us/step - loss: 8.2512
Epoch 1/1
2500/2500 [==============================] - 0s 188us/step - loss: 8.2045
Epoch 1/1
2500/2500 [==============================] - 0s 119us/step - loss: 8.1592
Epoch 1/1
2500/2500 [==============================] - 0s 114us/step - loss: 8.1182
Epoch 1/1
2500/2500 [==============================] - 0s 158us/step - loss: 8.0538
Epoch 1/1
2500/2500 [==============================] - 0s 179us/step - loss: 8.0079
Epoch 1/1
2500/2500 [==============================] - 0s 92us/step - loss: 7.9730


In [388]:
T = 10
yt_hat_std = np.array([np.exp(model_sm.predict([x_test, y_test])) for _ in range(T)])
H_x_test_std = utilities.optimal_h(yt_hat_std,loss_mat) 
acc_std = accuracy_score(y_test, H_x_test_std)
loss_std = np.mean(loss_mat[y_test, H_x_test_std])
print('Standard:\n')
print('Accuracy on optimal decision: ', acc_std)
print('Expected loss: ', loss_std)

Standard:

Accuracy on optimal decision:  0.6255
Expected loss:  0.36708999999999997


In [753]:
# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.
importlib.reload(new_custom_layers)
importlib.reload(utilities)

scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)

x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')

logits = new_custom_layers.OVELayer( 
    5,
    10, 
#     util_mat,
    kernel_divergence_fn=scaled_kl)([x_input, y_input])

model_sm = tf.keras.Model([x_input, y_input], logits)
model_sm.compile(loss=None, optimizer='adam')

In [750]:
history_epoch = model_sm.fit([x_train, y_train],
             batch_size=batch_size,
             epochs=10,
             verbose=1)
#                  validation_data=([x_val, y_val], None))
#                  callbacks=[model_checkpoint])
#     T = 10
#     yt_hat = np.array([lc_model.predict([x_train,H_x]) for _ in range(T)])
#     H_x = lcbnn.optimal_h(yt_hat,loss_mat)
#     h_train = np.argmax(model.predict([x_train, y_train, h_train]), -1)
#     history_model_sm.append(history_epoch)
       

Epoch 1/10
2500/2500 [==============================] - 0s 46us/step - loss: 8.7206
Epoch 2/10
2500/2500 [==============================] - 0s 54us/step - loss: 8.6919
Epoch 3/10
2500/2500 [==============================] - 0s 45us/step - loss: 8.6384
Epoch 4/10
2500/2500 [==============================] - 0s 50us/step - loss: 8.6044
Epoch 5/10
2500/2500 [==============================] - 0s 57us/step - loss: 8.5519
Epoch 6/10
2500/2500 [==============================] - 0s 54us/step - loss: 8.4532
Epoch 7/10
2500/2500 [==============================] - 0s 50us/step - loss: 8.5448
Epoch 8/10
2500/2500 [==============================] - 0s 43us/step - loss: 8.5322
Epoch 9/10
2500/2500 [==============================] - 0s 38us/step - loss: 8.3514
Epoch 10/10
2500/2500 [==============================] - 0s 46us/step - loss: 8.3240


In [751]:
T = 10
yt_hat_std = np.array([np.exp(model_sm.predict([x_test, y_test])) for _ in range(T)])
H_x_test_std = utilities.optimal_h(yt_hat_std,loss_mat) 
acc_std = accuracy_score(y_test, H_x_test_std)
loss_std = np.mean(loss_mat[y_test, H_x_test_std])
print('Standard:\n')
print('Accuracy on optimal decision: ', acc_std)
print('Expected loss: ', loss_std)

Standard:

Accuracy on optimal decision:  0.549
Expected loss:  0.44299000000000005


In [752]:
loss_mat

array([[0. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 0. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 0. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [0.7, 0.7, 0.7, 0. , 0.7, 0.7, 0.7, 0.7, 0.7, 0.7],
       [1. , 1. , 1. , 1. , 0. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 0. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 0. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 0. , 1. , 1. ],
       [0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0. , 0.7],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 0. ]])

In [439]:
loss_mat

array([[0. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 0. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 0. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [0.7, 0.7, 0.7, 0. , 0.7, 0.7, 0.7, 0.7, 0.7, 0.7],
       [1. , 1. , 1. , 1. , 0. , 1. , 1. , 1. , 1. , 1. ],
       [0.7, 0.7, 0.7, 0.7, 0.7, 0. , 0.7, 0.7, 0.7, 0.7],
       [1. , 1. , 1. , 1. , 1. , 1. , 0. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 0. , 1. , 1. ],
       [0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0. , 0.7],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 0. ]])

In [66]:
# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.
importlib.reload(new_custom_layers)
importlib.reload(utilities)

scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)

x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')
h_input =  tf.keras.Input((1,), name = 'h_input')

logits = custom_layers.SUCSoftmax( 
#     5,
    10, 
    util_mat,
    kernel_divergence_fn=scaled_kl)([x_input, y_input, h_input])

model_sm = tf.keras.Model([x_input, y_input, h_input], logits)
model_sm.compile(loss=None, optimizer='adam')

T = 1
h_train = y_train[:]
yt_hat = np.array([model_sm.predict([x_train, y_train, h_train]) for _ in range(T)])
h_train = utilities.optimal_h(yt_hat, loss_mat)

AttributeError: module 'custom_layers' has no attribute 'SUCSoftmax'

In [418]:
# history_model_sm_slc = []
for epoch in range(10):
    history_epoch = model_sm.fit([x_train, y_train, h_train],
             batch_size=batch_size,
             epochs=1,
             verbose=1)
    T = 1
    yt_hat = np.array([model_sm.predict([x_train, y_train, h_train]) for _ in range(T)])
    h_train = utilities.optimal_h(yt_hat, loss_mat)
#     h_train = model_sm_slc.predict([x_train, y_train, h_train])

Epoch 1/1
2500/2500 [==============================] - 0s 45us/step - loss: 7.6389
Epoch 1/1
2500/2500 [==============================] - 0s 49us/step - loss: 7.6159
Epoch 1/1
2500/2500 [==============================] - 0s 29us/step - loss: 7.5512
Epoch 1/1
2500/2500 [==============================] - 0s 56us/step - loss: 7.5797
Epoch 1/1
2500/2500 [==============================] - 0s 38us/step - loss: 7.3148
Epoch 1/1
2500/2500 [==============================] - 0s 51us/step - loss: 7.4312
Epoch 1/1
2500/2500 [==============================] - 0s 47us/step - loss: 7.2655
Epoch 1/1
2500/2500 [==============================] - 0s 41us/step - loss: 7.4425
Epoch 1/1
2500/2500 [==============================] - 0s 50us/step - loss: 7.2806
Epoch 1/1
2500/2500 [==============================] - 0s 43us/step - loss: 7.2133


In [419]:
T = 10
yt_hat_std = np.array([np.exp(model_sm.predict([x_test, y_test])) for _ in range(T)])
H_x_test_std = utilities.optimal_h(yt_hat_std,loss_mat) 
acc_std = accuracy_score(y_test, H_x_test_std)
loss_std = np.mean(loss_mat[y_test, H_x_test_std])
print('Standard:\n')
print('Accuracy on optimal decision: ', acc_std)
print('Expected loss: ', loss_std)

Standard:

Accuracy on optimal decision:  0.3521
Expected loss:  0.63515


In [8]:
rows = [[i] for i in range(num_classes)]
cols = [[i] for i in range(num_classes)]
vals = [[1] for i in range(num_classes)]

In [19]:
util_mat = 1 - np.eye(num_classes)
loss_mat = np.eye(num_classes)

In [20]:
util_mat

array([[0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 0., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 0., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 0., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 0., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 0., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 0., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 0., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.]])

In [143]:
# Scale the KL term from the probabilistic layers for per-datapoint ELBO. 
# KL term added internally as a regularization term.
importlib.reload(new_custom_layers)
importlib.reload(utilities)

scaled_kl = lambda q, p, _: tfp.distributions.kl_divergence(q, p) / tf.cast(train_samples, tf.float32)

x_input =  tf.keras.Input((x_train.shape[1:]), name = 'x_input')
y_input =  tf.keras.Input((1,), name = 'y_input')

logits = new_custom_layers.SUCSoftmax( 
#     5,
    num_classes, 
    rows, cols, vals,
#     util_mat,
    kernel_divergence_fn=scaled_kl)([x_input, y_input])

model_sm = tf.keras.Model([x_input, y_input], logits)
model_sm.compile(loss=None, optimizer='adam')

In [144]:
history_epoch = model_sm.fit([x_train, y_train],
         batch_size=batch_size,
         epochs=10,
         verbose=1)

Epoch 1/10
2500/2500 [==============================] - 6s 3ms/sample - loss: 10.4040
Epoch 2/10
2500/2500 [==============================] - 0s 150us/sample - loss: 10.1202
Epoch 3/10
2500/2500 [==============================] - 0s 76us/sample - loss: 9.9511
Epoch 4/10
2500/2500 [==============================] - 0s 74us/sample - loss: 9.8696
Epoch 5/10
2500/2500 [==============================] - 0s 102us/sample - loss: 9.7705
Epoch 6/10
2500/2500 [==============================] - 0s 97us/sample - loss: 9.6960
Epoch 7/10
2500/2500 [==============================] - 0s 56us/sample - loss: 9.6136
Epoch 8/10
2500/2500 [==============================] - 0s 78us/sample - loss: 9.5542
Epoch 9/10
2500/2500 [==============================] - 0s 79us/sample - loss: 9.4983
Epoch 10/10
2500/2500 [==============================] - 0s 68us/sample - loss: 9.4234


In [145]:
T = 10
yt_hat_std = np.array([np.exp(model_sm.predict([x_test, y_test])) for _ in range(T)])
H_x_test_std = utilities.optimal_h(yt_hat_std,loss_mat) 
acc_std = accuracy_score(y_test, H_x_test_std)
loss_std = np.mean(loss_mat[y_test, H_x_test_std])
print('Standard:\n')
print('Accuracy on optimal decision: ', acc_std)
print('Expected loss: ', loss_std)

Standard:

Accuracy on optimal decision:  0.7476
Expected loss:  0.2524


In [30]:
model_sm.predict([x_test, y_test])

array([[-0.887, -1.685, -0.058, ...,  2.011, -1.724,  0.278],
       [-2.706, -3.272,  2.064, ..., -3.967,  0.85 , -2.42 ],
       [-1.118,  1.594,  0.336, ..., -0.972, -0.248, -0.565],
       ...,
       [-0.244, -2.038, -1.755, ...,  0.011, -0.087,  0.439],
       [-0.543, -1.571, -0.893, ...,  1.038,  0.584,  0.42 ],
       [ 0.379, -1.515, -0.826, ...,  0.43 , -2.937, -1.965]],
      dtype=float32)

In [121]:
rows

[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]

In [122]:
cols

[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]

In [123]:
vals

[[1], [1], [1], [1], [1], [1], [1], [1], [1], [1]]